In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
# mat = pd.read_csv("matcsv.csv", header = None)
mat = np.genfromtxt('matcsv.csv', delimiter=',', dtype = int)

In [3]:
names = {
    0: "El Rosario",
    1: "Politécnico",
    2: "Instituto del Petróleo",
    3: "Indios Verdes",
    4: "Deportivo 18 de Marzo",
    5: "Martín Carrera",
    6: "La Raza",
    7: "Cuatro Caminos",
    8: "Tacuba",
    9: "Consulado",
    10: "Ciudad Azteca",
    11: "Guerrero",
    12: "Garibaldi",
    13: "Morelos",
    14: "Oceanía",
    15: "Hidalgo",
    16: "Bellas Artes",
    17: "Balderas",
    18: "Salto del Agua",
    19: "Pino Suárez",
    20: "Candelaria",
    21: "San Lázaro",
    22: "Tacubaya",
    23: "Centro Médico",
    24: "Chabacano",
    25: "Jamaica",
    26: "Pantitlán",
    27: "Mixcoac",
    28: "Zapata",
    29: "Ermita",
    30: "Atlalilco",
    31: "Barranca del Muerto",
    32: "Universidad",
    33: "Tasqueña",
    34: "Tláhuac"
}

In [4]:
adjacencies = {}

for i in range(len(mat)):
    adjacencies[i] = []
    for j in range(len(mat[0])):
        if(mat[i][j] != 10000):
            adjacencies[i].append(j)

In [5]:
adjacencies

{0: [2, 8],
 1: [2],
 2: [0, 1, 4, 6],
 3: [4],
 4: [2, 3, 5, 6],
 5: [4],
 6: [2, 4, 9, 11],
 7: [8],
 8: [0, 7, 15, 22],
 9: [6, 13, 14],
 10: [14],
 11: [6, 12, 15],
 12: [11, 13, 16],
 13: [9, 12, 20, 21],
 14: [9, 10, 21, 26],
 15: [8, 11, 16, 17],
 16: [12, 15, 18, 19],
 17: [15, 18, 22, 23],
 18: [16, 17, 19, 24],
 19: [16, 18, 20, 24],
 20: [13, 19, 21, 25],
 21: [13, 14, 20, 26],
 22: [8, 17, 27],
 23: [17, 24, 28],
 24: [18, 19, 23, 25, 29, 30],
 25: [20, 24, 26, 30],
 26: [14, 21, 25],
 27: [22, 28, 31],
 28: [23, 27, 29, 32],
 29: [24, 28, 30, 33],
 30: [24, 25, 29, 34],
 31: [27],
 32: [28],
 33: [29],
 34: [30]}

# Initial Population

In [6]:
population_size = 10
mutation_rate = 0.4
number_generations = 25

In [7]:
# def generate_population(adjacencies, population_size = 100):

#     populations = []

#     for i in range(population_size):

#         solution = [0]
#         forbidden = [1, 3, 5, 7, 10, 31, 32, 33, 34]

#         while(solution[-1] != 21):
#             next_node = random.choice(adjacencies[solution[-1]])
#             if (next_node in forbidden): 
#                 continue
#             else:
#                 solution.append(next_node)

#         populations.append(solution)
    
#     return populations

In [8]:
def generate_population(adjacencies, population_size = 100):

    populations = []

    for i in range(population_size):

        solution = [0]

        while(solution[-1] != 21):
            next_node = random.choice(adjacencies[solution[-1]])
            solution.append(next_node)

        populations.append(solution)
    
    return populations

In [9]:
population = generate_population(adjacencies, population_size)

# Fitness Function

In [10]:
def fitness(solution):
    
    current_index = 0
    distance = 0
    
    while(solution[current_index] != 21):
        start = solution[current_index]
        end = solution[current_index + 1]
        
        distance += mat[start][end]
        
        current_index += 1
    
    if(distance >= 10000):
        return 0
    else:
        return 1/distance

# Selection of Population

In [11]:
def selection(population):
    
    total_fitness = sum(fitness(solution) for solution in population)
    probabilities = [fitness(solution)/total_fitness for solution in population]
    selected = []
    
    for i in range(len(population)):
        chosen = random.choices(population, weights = probabilities)[0]
        selected.append(chosen)
    
    return selected

In [12]:
selected = selection(population)

# Crossover Function

In [13]:
def crossover(parent1, parent2):
    
    intersection = list(set(parent1) & set(parent2))
    
    proceed = False
    crossover_point = 0
    
    while(not proceed):
        crossover_point = random.randint(0, len(parent1)-1)
        if(parent1[crossover_point] in intersection):
            proceed = True
    
    value = parent1[crossover_point]
#     cut_point1 = len(parent2) - 1 - parent2[::-1].index(value)
#     cut_point2 = parent1.index(value)
    
    child1 = parent1[:crossover_point] + parent2[parent2.index(value):]
    child2 = parent2[:parent2.index(value)] + parent1[crossover_point:]
    
#     print(crossover_point, parent2.index(value))
    
    return child1, child2

# Mutation Rate

In [14]:
def mutation(solution, mutation_rate = 0.1):
     
    mutated_solution = solution.copy()
    
    for i in range(1,len(solution)-1):
        if random.random() < mutation_rate:
            mutated_solution[i] = random.choice(adjacencies[solution[i]])
            
    return mutated_solution

# Running of the Algorithm

In [15]:
for generation in range(number_generations):
    
    parents = selection(population)
    
    offspring = []
    for i in range(population_size//2):
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
        
        child1, child2 = crossover(parent1, parent2)
        
        offspring.append(child1)
        offspring.append(child2)
        
    offspring = [mutation(solution) for solution in offspring]
    
    population = offspring
    
    best_solution = max(population, key = fitness)
    
    print("Solution found in generation", generation+1)
    print("Action sequence:", best_solution)

Solution found in generation 1
Action sequence: [0, 2, 6, 11, 12, 13, 21]
Solution found in generation 2
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 3
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 4
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 5
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 6
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 7
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 8
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 9
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 10
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 11
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 12
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 13
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in generation 14
Action sequence: [0, 2, 6, 9, 14, 21]
Solution found in genera

In [16]:
d1 = 1/fitness([0, 8, 15, 8, 15, 17, 15, 11, 6, 9, 13, 20, 13, 9, 14, 21])
d2 = 1/fitness([0, 2, 6, 9, 13, 21])

In [17]:
print(d1, d2)

47.0 14.0


In [18]:
sol = [0, 2, 6, 9, 13, 21]
for estation in sol:
    print(names[estation])

El Rosario
Instituto del Petróleo
La Raza
Consulado
Morelos
San Lázaro
